In [1]:
import pandas as pd
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
tabelas = pd.read_html("https://mudancanorteblumenau.paginas.ufsc.br/logistica/area-dos-lideres-de-ambiente/lista-de-ambientes-e-lideres-designados/")

In [3]:
DADOS = []

for tabela in tabelas:
    edificio = tabela.iloc[0,0]
    cabecalho = tabela.iloc[1,:]
    dados = tabela.iloc[2:,:]
    dados.columns = cabecalho
    dados["edificio"] = edificio
    DADOS.append(dados)

/tmp/ipykernel_1677966/2592225809.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados["edificio"] = edificio
/tmp/ipykernel_1677966/2592225809.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados["edificio"] = edificio
/tmp/ipykernel_1677966/2592225809.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

In [4]:
DADOS = pd.concat(DADOS)

In [18]:
def trysplit(string: str, n: int):
    try:
        return string.split(" – ")[n].strip()
    except IndexError:
        return trysplit(string, n-1)

In [19]:
DADOS["a_codigo"] = DADOS.Ambiente.apply(lambda x: trysplit(x,0))
DADOS["a_nome"] = DADOS.Ambiente.apply(lambda x: trysplit(x,1))

In [20]:
DADOS

1,Ambiente,lider,edificio,a_codigo,a_nome
2,A001 – BIBLIOTECA SETORIAL DO CAMPUS BLUMENAU ...,Priscila Margarete Bona,BNU02 – Bloco Acadêmico A,A001,BIBLIOTECA SETORIAL DO CAMPUS BLUMENAU (Térreo)
3,A001.1 – ADMINISTRAÇÃO BIBLIOTECA (Térreo),Priscila Margarete Bona,BNU02 – Bloco Acadêmico A,A001.1,ADMINISTRAÇÃO BIBLIOTECA (Térreo)
4,A001.3 – AGENTE PATRIMONIAL (Térreo),Fabio Metzner,BNU02 – Bloco Acadêmico A,A001.3,AGENTE PATRIMONIAL (Térreo)
5,A001.3.1 – SALA DO SERVIDOR (Térreo),Fabio Metzner,BNU02 – Bloco Acadêmico A,A001.3.1,SALA DO SERVIDOR (Térreo)
6,A002 – SECRETARIA ACADÊMICA (Térreo),Cláudio de Morais,BNU02 – Bloco Acadêmico A,A002,SECRETARIA ACADÊMICA (Térreo)
...,...,...,...,...,...
65,CC01 – CORREDOR 1º ANDAR (1º Andar),Patrícia A. A. de F. Barthel,BNU07 – Bloco Administrativo C,CC01,CORREDOR 1º ANDAR (1º Andar)
66,CC02 – CORREDOR 2º ANDAR (2º Andar),Marilena Corrêa Fialho,BNU07 – Bloco Administrativo C,CC02,CORREDOR 2º ANDAR (2º Andar)
67,CC03 – CORREDOR 3º ANDAR (3º Andar),Ivan de Matos,BNU07 – Bloco Administrativo C,CC03,CORREDOR 3º ANDAR (3º Andar)
2,RU001 – Restaurante Universitário (Térreo),Edgar Noschang Kunz,BNU08 – Restaurante Universitário,RU001,Restaurante Universitário (Térreo)


In [21]:
DADOS.rename(columns={"Líder de Ambiente": "lider"}, inplace=True)

In [22]:
lideres = DADOS.lider.unique()

In [23]:
DATABASE = {}

for lider in lideres:
    DATABASE[lider] = list(DADOS.loc[DADOS.lider == lider, "a_codigo"]) 

In [ ]:
import json

#with open("json/lideres.json", "w") as f:
#    json.dump(DATABASE, f, indent=2, ensure_ascii=False)

In [24]:
full_ambientes = DADOS.a_codigo.unique()

In [25]:
from load_lideres import load_json_file

In [26]:
ambientes = pd.DataFrame(load_json_file("json/ambientes.json"))

In [28]:
JOIN = pd.merge(left=DADOS,right=ambientes, left_on="a_codigo", right_on="codigo", how="outer")

In [32]:
JOIN

,Ambiente,lider_x,edificio_x,a_codigo,a_nome,edificio_y,codigo,nome,items,lider_y,status,campus
0,A001 – BIBLIOTECA SETORIAL DO CAMPUS BLUMENAU ...,Priscila Margarete Bona,BNU02 – Bloco Acadêmico A,A001,BIBLIOTECA SETORIAL DO CAMPUS BLUMENAU (Térreo),BNU02 - Bloco Acadêmico A,A001,BIBLIOTECA SETORIAL DO CAMPUS BLUMENAU (Térreo),256.0,None,Em uso,Sul
1,A001.1 – ADMINISTRAÇÃO BIBLIOTECA (Térreo),Priscila Margarete Bona,BNU02 – Bloco Acadêmico A,A001.1,ADMINISTRAÇÃO BIBLIOTECA (Térreo),BNU02 - Bloco Acadêmico A,A001.1,ADMINISTRAÇÃO BILBIOTECA (Térreo),27.0,None,Em uso,Sul
2,A001.3 – AGENTE PATRIMONIAL (Térreo),Fabio Metzner,BNU02 – Bloco Acadêmico A,A001.3,AGENTE PATRIMONIAL (Térreo),BNU02 - Bloco Acadêmico A,A001.3,AGENTE PATRIMONIAL (Térreo),8.0,None,Em uso,Sul
3,A001.3.1 – SALA DO SERVIDOR (Térreo),Fabio Metzner,BNU02 – Bloco Acadêmico A,A001.3.1,SALA DO SERVIDOR (Térreo),BNU02 - Bloco Acadêmico A,A001.3.1,SALA DO SERVIDOR (Térreo),4.0,None,Em uso,Sul
4,A002 – SECRETARIA ACADÊMICA (Térreo),Cláudio de Morais,BNU02 – Bloco Acadêmico A,A002,SECRETARIA ACADÊMICA (Térreo),BNU02 - Bloco Acadêmico A,A002,SECRETARIA ACADÊMICA (Térreo),58.0,None,Em uso,Sul
...,...,...,...,...,...,...,...,...,...,...,...,...
182,CC01 – CORREDOR 1º ANDAR (1º Andar),Patrícia A. A. de F. Barthel,BNU07 – Bloco Administrativo C,CC01,CORREDOR 1º ANDAR (1º Andar),BNU07 - Bloco Administrativo C,CC01,CORREDOR 1º ANDAR (1º Andar),6.0,None,Em uso,Sul
183,CC02 – CORREDOR 2º ANDAR (2º Andar),Marilena Corrêa Fialho,BNU07 – Bloco Administrativo C,CC02,CORREDOR 2º ANDAR (2º Andar),BNU07 - Bloco Administrativo C,CC02,CORREDOR 2º ANDAR (2º Andar),5.0,None,Em uso,Sul
184,CC03 – CORREDOR 3º ANDAR (3º Andar),Ivan de Matos,BNU07 – Bloco Administrativo C,CC03,CORREDOR 3º ANDAR (3º Andar),BNU07 - Bloco Administrativo C,CC03,CORREDOR 3º ANDAR (3º Andar),5.0,None,Em uso,Sul
185,RU001 – Restaurante Universitário (Térreo),Edgar Noschang Kunz,BNU08 – Restaurante Universitário,RU001,Restaurante Universitário (Térreo),BNU08 - Restaurante Universitário,RU001,Restaurante Universitário (Térreo),5.0,None,Em uso,Sul


In [43]:
AMBIENTES_VAZIOS = []
for row in JOIN[JOIN.codigo.isna()].itertuples(): 
    dados = {
    "edificio":row.edificio_x.replace("–", "-"),
    "codigo": row.a_codigo,
    "nome":row.a_nome,
    "items": 0,
    "lider": None,
    "status":"Em uso",
    "campus":"Sul"
  }
    AMBIENTES_VAZIOS.append(dados)

In [46]:
ambientes_vazios = pd.DataFrame(AMBIENTES_VAZIOS)

In [47]:
ambientes_vazios

,edificio,codigo,nome,items,lider,status,campus
0,BNU02 - Bloco Acadêmico A,A006,SANITÁRIO PCD (Térreo),0,None,Em uso,Sul
1,BNU02 - Bloco Acadêmico A,A007,SANITÁRIO FEMININO (Térreo),0,None,Em uso,Sul
2,BNU02 - Bloco Acadêmico A,A008,SANITÁRIO MASCULINO (Térreo),0,None,Em uso,Sul
3,BNU02 - Bloco Acadêmico A,A109,SANITÁRIO FEMININO (1º Andar),0,None,Em uso,Sul
4,BNU02 - Bloco Acadêmico A,A111,SANITÁRIO MASCULINO (1º Andar),0,None,Em uso,Sul
5,BNU02 - Bloco Acadêmico A,A209,SANITÁRIO PDC FEMININO (2º Andar),0,None,Em uso,Sul
6,BNU02 - Bloco Acadêmico A,A210,SANITÁRIO PCD FEMININO (2º Andar),0,None,Em uso,Sul
7,BNU02 - Bloco Acadêmico A,A212,SANITÁRIO MASCULINO (2º Andar),0,None,Em uso,Sul
8,BNU02 - Bloco Acadêmico A,A214,SANITÁRIO MASCULINO (2º Andar),0,None,Em uso,Sul
9,BNU02 - Bloco Acadêmico A,A307,CORREDOR ACESSO A307.1 E A301.2 (3º Andar),0,None,Em uso,Sul


In [53]:
full_ambientes  = pd.concat([ambientes, ambientes_vazios]).sort_values(by="codigo")


In [54]:
full_ambientes.to_json("json/ambientes.json",  orient="records", indent=2)

In [ ]:
no_sistema = [x['codigo'] for x in ambientes]

In [ ]:
len(full_ambientes)

187

In [ ]:
len(no_sistema)

136

In [ ]:
set(full_ambientes) - set(no_sistema)

{'A006',
 'A007',
 'A008',
 'A109',
 'A111',
 'A209',
 'A210',
 'A212',
 'A214',
 'A307',
 'A308',
 'A309',
 'A311',
 'A312',
 'B006',
 'B007',
 'B009',
 'B101',
 'B102',
 'B108',
 'B109',
 'C002',
 'C101.1',
 'C102.1',
 'C104.1',
 'C105',
 'C105.3.1',
 'C106.1',
 'C109',
 'C201.1',
 'C202.1',
 'C203.1',
 'C204.2',
 'C204.3',
 'C204.4',
 'C204.5',
 'C204.7',
 'C205.1',
 'C208',
 'C302.1',
 'C303.1',
 'C304.1',
 'C305.1',
 'C306.1',
 'C309',
 'C310',
 'C401.1',
 'CC00',
 'Pantográfica',
 'SALA INTEGRE JUNIOR',
 'SANITÁRIO INTEGRE JUNIOR'}